## Скачиваем геном близкородственной бактерии T.oleivorans

In [3]:
!sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.


To activate EDirect for this terminal session, please execute the following:

export PATH=${HOME}/edirect:${PATH}



In [2]:
!echo "export PATH=/root/edirect:\${PATH}" >> ${HOME}/.bashrc

In [4]:
!export PATH=${HOME}/edirect:${PATH}

In [5]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk

## Аннотация генома

In [6]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.1 MB/s eta 0:00:00


In [7]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from Bio.Data import CodonTable
from Bio import Entrez
from Bio.SeqFeature import SeqFeature, FeatureLocation
from datetime import datetime as dt
import pandas as pd

In [8]:
# Посмотрим содержание файла до заполнения
scaffolds = []
for scaf in SeqIO.parse('after_gap.fa', format='fasta'):
  scaffolds.append(scaf)

# В формате словаря annotations и features пустые.
# Будем заполнять их

scaffolds[0].__dict__

{'_seq': Seq('TAAAACCCCCTTCCGGTATCGGTTGGGGGTTTTTTCTTGGCTGCTATTTGATGC...TTA'),
 'id': 'scaffold1_cov231',
 'name': 'scaffold1_cov231',
 'description': 'scaffold1_cov231',
 'dbxrefs': [],
 'annotations': {},
 '_per_letter_annotations': {},
 'features': []}

## Сначала заполним общую информацию в annotations и id скаффолда

In [9]:
# Сначала заполним общую информацию в annotations и id скаффолда
scaffolds = {}
for scaf in SeqIO.parse("after_gap.fa", "fasta"):

  scaf.annotations['molecule_type'] = 'DNA'
  scaf.annotations['data_file_division'] = 'BCT'
  scaf.annotations['date'] = dt.now().strftime("%d-%b-%Y").upper()

  scaffolds[scaf.id] = scaf

In [10]:
# Просмотрим, записалась ли информация

scaffolds['scaffold10_cov287'].__dict__

{'_seq': Seq('AGACCCGCTTTTTCGATATTGCTAAAATCAATTCCGGCACCATTGAGTGCATTG...CGC'),
 'id': 'scaffold10_cov287',
 'name': 'scaffold10_cov287',
 'description': 'scaffold10_cov287',
 'dbxrefs': [],
 'annotations': {'molecule_type': 'DNA',
  'data_file_division': 'BCT',
  'date': '22-OCT-2023'},
 '_per_letter_annotations': {},
 'features': []}

## Заполняем features

### Работа с proteins.fasta (позиции + трансляция)

In [11]:
# Заполним координаты генов, стренд и тип по информации из файла proteins.fasta

prot = {}
for g in SeqIO.parse("proteins.fasta", "fasta"):
  inf = g.description.split(' ')

  if inf[4] == '+':
    strand = 1
  else:
    strand = -1

  feat = SeqFeature(FeatureLocation(int(inf[2]), int(inf[3]), strand=strand), type="CDS")
  feat.qualifiers['locus_tag'] = [f'LT_{inf[0]}']

# И трансляции генов

  feat.qualifiers['translation'] = [g.seq]

  scaffolds[inf[1]].features.append(feat)

  prot[inf[0]] = feat

In [12]:
# Проверяем, заполнились ли features

feat.qualifiers

{'locus_tag': ['LT_3614'],
 'translation': [Seq('MLKANNLTVSMSRRGNCHDNACAESFFALLKRERIRRKIYRTREEGKADIFNYI...RHG')]}

### Добавляем функции белков

In [13]:
# Ищем и добавляем все функции из файла T_oleivorans_MIL_1.gbk

mil = {}
for feat in SeqIO.read("T_oleivorans_MIL_1.gbk", "genbank").features:
  if ('protein_id' not in feat.qualifiers) or ('product' not in feat.qualifiers):
    continue

  mil[feat.qualifiers['protein_id'][0]] = feat.qualifiers['product'][0]

mil1_hits = pd.read_csv('scaffolds.hits_from_MIL_1.txt', sep='\t', header=None, names=['qseqid', 'sseqid', '0', '1', '2', '3', '4', '5', '6', '7', '8', 'bitscore'])

In [14]:
hits = mil1_hits[mil1_hits['sseqid'].str.contains("CCU")].copy()
hits.sort_values('bitscore', ascending=False, inplace=True)
hits.drop_duplicates('qseqid', inplace=True)

for i, hit in hits.iterrows():
  gene = prot[str(hit['qseqid'])]
  gene.qualifiers['product'] = [mil[hit['sseqid'].split('_')[2]]]

In [15]:
# Сохраняем все записи в файл формата genbank

SeqIO.write(scaffolds.values(), "GENOME.gbk", "genbank")

67

In [17]:
# Смотрим, как заполнился файл

!tail -n200 GENOME.gbk

VERSION     scaffold59_cov472
KEYWORDS    .
SOURCE      .
  ORGANISM  .
            .
FEATURES             Location/Qualifiers
     CDS             complement(3..703)
                     /locus_tag="LT_3606"
                     /translation="MIKRIYHLSNGSAGARTIATASTTEGLPMTRYIASKRMKELNFVS
                     CQLPNHSYKRGGNEHVSIANTLNRQFKVEQPNQVWCGDVTYIWTGKRWAYLAVVLDLYA
                     RQVVGWAMSHSPDSELTTKALHLAFEARGRPKNLMFHSDQGCHYTSLKFRQTLWRLQIQ
                     QSMSRRGNCWDNAPMERFFRSFKTEWMPSTGYQSFAEARSEVSRYITDYYSRYRPHTFN
                     GGLTPAEAEARY"
                     /product="transposase orfB"
     CDS             complement(833..1170)
                     /locus_tag="LT_3607"
                     /translation="MPSYIYGDNMTKQTRPRFNPEFKVEAAQLVLDQGYSVKEAAQAMG
                     VGLSTLDKWVRKLKNERDGQLTPGNPITPEQREIAELKKQVKRLELEKEILKKASALLM
                     SDSMNGLR"
                     /product="transposase orfA"
ORIGIN
        1 tatatcttgc ctcagcttca gcgggtgtta acccgc